In [ ]:
input = read("day15.txt", String)

whouse = split(input, "\n\n")[1]
rmoves = split(input, "\n\n")[2]

row_length = length(split(whouse,"\n")[1])

function step_robot(string, move)
   if move == only(">")
       m = match(r"@(O*?)\.", string)  
        if !isnothing(m)
            return(replace(string, m.match => ".@"*m.captures[1]))
        end 
    elseif move == only("<")
       m = match(r"\.(O*?)@", string)  
        if !isnothing(m)
            return(replace(string, m.match => m.captures[1] * "@."))
        end 
    elseif move == only("^")
        m = match(Regex("\\.(.{$(row_length)})@", "s"), string)
        if !isnothing(m)
            return(replace(string, m.match => "@" * m.captures[1] * "."))
        end
        m = match(Regex("\\.((?:.{$(row_length)}O)+?)(.{$(row_length)})@", "s"), string)
        if !isnothing(m)
            return(replace(string, m.match => "O" * m.captures[1][1:end-1] * "@" * m.captures[2] * "."))
        end 
    elseif move == only("v")
        m = match(Regex("@(.{$(row_length)})\\.", "s"), string)
        if !isnothing(m)
            return(replace(string, m.match => "." * m.captures[1] * "@"))
        end
        m = match(Regex("@(.{$(row_length)})((?:O.{$(row_length)})+?)\\.", "s"), string)
        if !isnothing(m)
            return(replace(string, m.match => "." * m.captures[1] * "@" * m.captures[2][2:end] * "O"))
        end 
    end
    return(string)
end

function move_robot(string, moves)
   for c in moves
        string = step_robot(string, c)
    end
    return(string)
end

function scores(string, row_length)
     [(fld(m.offset, (row_length+1))*100 + mod(m.offset, (row_length+1))-1) for m in eachmatch(r"O", string)]
end

#println(whouse)
#println(move_robot(whouse, rmoves))
@time println(sum(scores(move_robot(whouse, rmoves), row_length)))


In [ ]:
input2 = replace(input, "O" => "LR")
input2 = replace(input2, "#" => "##")
input2 = replace(input2, "." => "..")
input2 = replace(input2, "@" => "@.")

#input2 = read("day15test2.txt", String)

whouse2 = split(input2, "\n\n")[1]
rmoves2 = split(input2, "\n\n")[2]

row_length2 = length(split(whouse2,"\n")[1])


function step_robot2(string, move)
   if move == only(">")
       m = match(r"@((?:LR)*)\.", string)  
        if !isnothing(m)
            return(replace(string, m.match => ".@"*m.captures[1]))
        end 
        
    elseif move == only("<")
        m = match(r"\.((?:LR)*)@", string)  
        if !isnothing(m)
            return(replace(string, m.match => m.captures[1] * "@."))
        end 

    elseif move == only("^")
        new_string = ""
        # find the rocks which are being pushed
        # and turn them in to NM instead of LR
        keepgoing = true
        while keepgoing
            new_string = replace(string, Regex("LR(?=.{$(row_length2)}[@NM])", "s") => "NM")
            new_string = replace(new_string, Regex("LR(?=.{$(row_length2-1)}[@NM])", "s") => "NM")
            if new_string == string
                keepgoing = false
            end
            string = new_string
        end
        
       # but then turn back the NMs that have walls above them
       keepgoing = true
       while keepgoing
           new_string = replace(string, Regex("(?<![.NM]{2}.{$(row_length2-1)})NM", "s") => "LR")
           if new_string == string
               keepgoing = false
           end
           string = new_string
       end
        
        # check if after all this that the robot can still move
        if occursin(Regex("[.MN](.{$(row_length2)})@", "s"), string)
                   
            # if so then move the NMs that have dots above
            keepgoing = true
            while keepgoing
                m = match(Regex("[.]{2}(.{$(row_length2-1)})NM", "s"), string)
                if !isnothing(m)
                    new_string = replace(string, m.match => "LR" * m.captures[1] * "..")
                end
                if new_string == string
                    keepgoing = false
                end
                string = new_string
            end
            
            # then move the robot
            m = match(Regex("\\.(.{$(row_length2)})@", "s"), string)
            if !isnothing(m)
                string = replace(string, m.match => "@" * m.captures[1] * ".")
            end
        else
            # if not then turn the NMs back to LRs which were hanging
            string = replace(string, "NM" => "LR")  
        end
        
        return(string)

    elseif move == only("v")
        new_string = ""
        # find the rocks which are being pushed
        # and turn them in to NM instead of LR
        keepgoing = true
        while keepgoing
            new_string = replace(string, Regex("(?<=[@NM].{$(row_length2)})LR", "s") => "NM")
            new_string = replace(new_string, Regex("(?<=[@NM].{$(row_length2-1)})LR", "s") => "NM")
            if new_string == string
                keepgoing = false
            end
            string = new_string
        end
        
        # but then turn back the NMs that have walls beneath them
        keepgoing = true
        while keepgoing
            new_string = replace(string, Regex("NM(?!.{$(row_length2-1)}[.NM]{2})", "s") => "LR")
            if new_string == string
                keepgoing = false
            end
            string = new_string
        end
        
        # check the robot can still move after all this
        if occursin(Regex("@(.{$(row_length2)})[.MN]", "s"), string)
            
            # if so then move the NMs that have dots below
            keepgoing = true
            while keepgoing
                m = match(Regex("NM(.{$(row_length2-1)})[.]{2}", "s"), string)
                if !isnothing(m)
                    new_string = replace(string, m.match => ".." * m.captures[1] * "LR")
                end
                if new_string == string
                    keepgoing = false
                end
                string = new_string
            end
            
            # then move the robot
            m = match(Regex("@(.{$(row_length2)})\\.", "s"), string)
            if !isnothing(m)
                string = replace(string, m.match => "." * m.captures[1] * "@")
            end
        else
            string = replace(string, "NM" => "LR")
        end
    end
    return(string)
end

function move_robot2(string, moves)
   for c in moves
        string = step_robot2(string, c)
        if occursin("N", string); println("N") end
        if occursin("M", string); println("M") end
        #println(c)
        #println(string)
        #IJulia.clear_output(true)
        #sleep(0.5)
    end
    return(string)
end

function scores2(string, row_length)
     [(fld(m.offset, (row_length+1))*100 + mod(m.offset, (row_length+1))-1) for m in eachmatch(r"L", string)]
end


println(whouse2)
@time whouse_end2 = move_robot2(whouse2,rmoves2)
println(whouse_end2)
println(sum(scores2(whouse_end2, row_length2)))
